In [133]:
from datetime import datetime

def getExpenseDateFromUser():
    counter = 10

    # give a few attempts to get the date from the user
    while(counter > 0):
        user_date = input("Please enter the date of the expense in the format YYYY-MM-DD:")
        try:
            date = datetime.strptime(user_date, '%Y-%m-%d').date()
            return date
        except ValueError:
            print("Invalid date format. Please use YYYY-MM-DD")
            counter -= 1

    raise ValueError("Invalid date format.")

def getCategoryExpenseFromUser():
    user_category = input("Please enter the expense category:")
    return user_category

def getAmountSpentFromUser():
    counter = 10

    # give a few attempts to get the amount from the user
    while(counter > 0):
        user_amount = input("Please enter the amount spent:")
        try:
            amount = float(user_amount)
            return amount
        except ValueError:
            print(f"Invalid value for amount: {user_amount}")
            counter -= 1

    raise ValueError("Invalid value for amount. Must be a number.")

def getDescriptionOfExpenseFromUser():
    user_description = input("Please enter a brief description of expense:")
    return user_description

def getExpenseFromUser():
    expense_date = ""
    try:
        expense_date = getExpenseDateFromUser()
    except ValueError: 
        print("Unable to get expense date.")

    expense_category = getCategoryExpenseFromUser()

    expense_amount = 0
    try:
        expense_amount = getAmountSpentFromUser()
    except ValueError:
        print("Unable to get expense amount.")

    expense_description = getDescriptionOfExpenseFromUser()

    expense_dict = {
        "date": expense_date,
        "category": expense_category,
        "amount": expense_amount,
        "description": expense_description
    }
    
    return expense_dict

In [135]:
def addExpenseToList(expense_list, expense_dict):
    if not expense_list or type(expense_list) != list:
        print("Invalid expense list")
        return
    
    if not expense_dict or type(expense_dict) != dict:
        print("Invalid expense dictionary")
        return

    expense_list.append(expense_dict)
    return expense_list

def viewExpenses(expense_list=[]):
    if not expense_list or type(expense_list) != list:
        print("Invalid expense list")
        return
    
    for expense in expense_list:
        if not expense or type(expense) != dict:
            print(f"Invalid expense: {expense}")
            continue

        if 'date' in expense and expense['date'] != "":
            print(f"Date: {expense['date']}")
        else:
            print(f"No date found for expense {expense}")
        if 'category' in expense:
            print(f"Category: {expense['category']}")
        else:
            print(f"No category found for expense {expense}")
        if 'amount' in expense and expense['amount'] != 0:
            print(f"Amount: {expense['amount']}")
        else:
            print(f"No amount found for expense {expense}")
        if 'description' in expense:
            print(f"Description: {expense['description']}")
        else:
            print(f"No description found for expense {expense}")
        print("--------------------------------")

In [136]:
def getBudgetFromUser():
    counter = 10

    while(counter > 0):
        user_budget = input("Please enter the amount of your budget:")
        try:
            budget = float(user_budget)
            return budget
        except ValueError:
            print(f"Invalid value for budget: {user_budget}")
            counter -= 1

    raise ValueError("Invalid string for float conversion")

def getTotalExpenses(expense_list):
    if not expense_list or type(expense_list) != list:
        return 0

    total_expenses = 0
    for expense in expense_list:
        if not expense or type(expense) != dict:
            continue

        # check if expense is in current month
        if expense['date'].month == datetime.now().month:
            if 'amount' in expense:
                total_expenses += expense['amount']
    
    return total_expenses
    
def getRemainingBudget(budget, total_expenses):
    return budget - total_expenses


In [137]:
import csv
import os
from itertools import chain

def saveExpensesToCSV(expense_list, filename):
    if not expense_list or type(expense_list) != list:
        print("Invalid expense list")
        return

    with open(filename, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(['date', 'category', 'amount', 'description'])
        for expense in expense_list:
            if not expense or type(expense) != dict:
                continue

            # only write expenses that are valid with all required fields
            if 'date' not in expense or expense['date'] == "":
                continue
            if 'category' not in expense:
                continue
            if 'amount' not in expense or expense['amount'] == 0:
                continue
            if 'description' not in expense:
                continue
            
            writer.writerow([expense['date'].strftime('%Y-%m-%d'), expense['category'], expense['amount'], expense['description']])

def expenseRowReader(filename):
    if not os.path.exists(filename):
        return

    with open(filename, 'r') as f:
        reader = csv.reader(f)
        for i, row in enumerate(reader):
            if i == 0: # skip header row
                continue

            yield row

def loadExpensesFromCSV(filename):
    loaded_expenses = []

    for row in expenseRowReader(filename):
        expense_dict = {
            'date': datetime.strptime(row[0], '%Y-%m-%d').date(), # convert date string to date object
            'category': row[1],
            'amount': float(row[2]),
            'description': row[3]
        }
        loaded_expenses = chain(loaded_expenses, [expense_dict])

    return list(loaded_expenses)

In [ ]:
def getMenuOptionFromUser():
    option = 0
    print("Please select an option:")
    print("1. Add an expense")
    print("2. View expenses")
    print("3. Track your budget")
    print("4. Save expenses")
    print("5. Exit")
    try:
        option = int(input("Enter your choice: "))
    except ValueError:
        option = 0
    return option

def mainMenu():
    expense_list = loadExpensesFromCSV('expenses.csv')

    option = 0
    while(option != 5):
        option = getMenuOptionFromUser()
        if option == 1:
            expense_dict = getExpenseFromUser()
            expense_list = addExpenseToList(expense_list, expense_dict)
        elif option == 2:
            viewExpenses(expense_list)
        elif option == 3:
            try:
                budget = getBudgetFromUser()
            except ValueError:
                print("Unable to get budget. Please try again.")
                option = 0
                continue
            total_expenses = getTotalExpenses(expense_list)
            remaining_budget = getRemainingBudget(budget, total_expenses)
            print(f"You have spent {total_expenses} of your budget of {budget}.")
            if remaining_budget < 0:
                print("Warning: You have exceeded your budget")
            else:
                print(f"You have {remaining_budget} left in your budget.")
        elif option == 4:
            saveExpensesToCSV(expense_list, 'expenses.csv')
        elif option == 5:
            print("Exiting program...")
            return
        else:
            print("Invalid option. Please try again.")

def main():
    mainMenu()

if __name__ == "__main__":
    main()
